<a href="https://colab.research.google.com/github/positivejmk/Tobigs_17/blob/main/Tobigs_Week3_DT_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DT Assignment

# Data Loading

In [1]:
import pandas as pd 
import numpy as np

In [4]:
pd_data = pd.read_csv('https://raw.githubusercontent.com/AugustLONG/ML01/master/01decisiontree/AllElectronics.csv')
pd_data.drop("RID",axis=1, inplace = True) #RID는 그냥 순서라서 삭제
pd_data

,age,income,student,credit_rating,class_buys_computer
0,youth,high,no,fair,no
1,youth,high,no,excellent,no
2,middle_aged,high,no,fair,yes
3,senior,medium,no,fair,yes
4,senior,low,yes,fair,yes
5,senior,low,yes,excellent,no
6,middle_aged,low,yes,excellent,yes
7,youth,medium,no,fair,no
8,youth,low,yes,fair,yes
9,senior,medium,yes,fair,yes


## Gini 계수를 구하는 함수 만들기

<img src="gini.png" width="200">

- Input: df(데이터), label(타겟변수명)
- 해당 결과는 아래와 같이 나와야 합니다.

In [5]:
def get_gini(df, label):
    
    # df 데이터 개수
    n = df.shape[0]

    # label의 정답 값
    label_answer = np.unique(df[label])

    # df의 label 컬럼 중 첫번째 레이블 수 (no일때)
    label1 = np.sum(df[label] == label_answer[0])

    # df의 label 컬럼 중 두번째 레이블 수 (yes일때)
    label2 = n - label1

    # 지니계수 구하기
    gini = 1 - (label1/n) **2 - (label2/n) **2
    
    return gini

``` Testing ```

In [6]:
pd_data.shape

(14, 5)

In [7]:
pd_data.shape[0]

14

In [8]:
np.unique(pd_data['class_buys_computer'])

array(['no', 'yes'], dtype=object)

``` Result ```

In [9]:
get_gini(pd_data,'class_buys_computer')

0.4591836734693877

## Feature의 Class를 이진 분류로 만들기
- ex) {A,B,C} -> ({A}, {B,C}), ({B}, {A,C}), ({C}, {A,B})

- Input: df(데이터), attribute(Gini index를 구하고자 하는 변수명)
- Income 변수를 결과로 출력해주세요.

In [10]:
from itertools import combinations

def get_binary_split(df, attribute):
    
    uniques = list(df[attribute].unique()) # 속성 데이터 고유값들을 담은 리스트 
    result=[]
    
    # 데이터 고유값 -1 만큼의 조합 생성
    for i in range(1, len(uniques)):
      for att in combinations(uniques, i):
        result.append(list(att))

    return result

``` Testing ```

In [11]:
list(pd_data['income'].unique())

['high', 'medium', 'low']

In [12]:
pd_data['income'].unique()

array(['high', 'medium', 'low'], dtype=object)

In [13]:
np.unique(pd_data['income'])

array(['high', 'low', 'medium'], dtype=object)

In [14]:
c = combinations((['high', 'low', 'medium']),2)

In [15]:
print(list(c))

[('high', 'low'), ('high', 'medium'), ('low', 'medium')]


``` Result ```

In [16]:
get_binary_split(pd_data,'income')

[['high'],
 ['medium'],
 ['low'],
 ['high', 'medium'],
 ['high', 'low'],
 ['medium', 'low']]

## 모든 이진분류의 경우의 Gini index를 구하는 함수 만들기
- 위에서 완성한 두 함수를 사용하여 만들어주세요!
- 해당 결과는 아래와 같이 나와야 합니다.

In [18]:
def get_attribute_gini_index(df, attribute, label):
    # DataFrame의 row 갯수
    n = df.shape[0]
    
    # 우선 이진 분류로 만들기
    split = get_binary_split(df, attribute)
    
    # 결과를 담을 dictionary 선언
    result = {}
    
    # split에 담겨있는 원소 하나씩 수행
    for col in split:
        # dictionary에 담을 이름 선언(2개 이상일시 _로 구분되고 1개일 시 그냥 들어갈 것.)
        col_name = "_".join(col)
        
        # 뒤집기 연산 (~)을 하기위해 numpy array 형식으로 지정.
        mask = np.array(list(map(lambda x : x in col, df[attribute])))
        
        # 원하는 DataFrame만 뽑아주기(t = target, r = rest)
        target = df.loc[mask, :]
        rest = df.loc[~mask, :]        

        # gini계수 계산
        gini_t = get_gini(target, label)
        gini_r = get_gini(rest, label)
        
        # 가중치 분배하여 최종 gini계수 계산
        gini = len(target) / n * gini_t + len(rest) / n * gini_r
        
        # 생성해놨던 dictionary에 추가
        result[col_name] = gini
        
    return result

``` Testing ```

In [19]:
# income 이 high일 경우만 예시로 테스트
mask = np.array(list(map(lambda x: x in ['high'], pd_data['income'])))

In [20]:
print(mask)

[ True  True  True False False False False False False False False False
  True False]


In [21]:
pd_data.loc[mask, :]

,age,income,student,credit_rating,class_buys_computer
0,youth,high,no,fair,no
1,youth,high,no,excellent,no
2,middle_aged,high,no,fair,yes
12,middle_aged,high,yes,fair,yes


In [22]:
pd_data.loc[~mask, :]

,age,income,student,credit_rating,class_buys_computer
3,senior,medium,no,fair,yes
4,senior,low,yes,fair,yes
5,senior,low,yes,excellent,no
6,middle_aged,low,yes,excellent,yes
7,youth,medium,no,fair,no
8,youth,low,yes,fair,yes
9,senior,medium,yes,fair,yes
10,youth,medium,yes,excellent,yes
11,middle_aged,medium,no,excellent,yes
13,senior,medium,no,excellent,no


In [23]:
get_gini(pd_data.loc[mask, :], 'class_buys_computer')

0.5

In [24]:
get_gini(pd_data.loc[~mask, :], 'class_buys_computer')

0.4200000000000001

``` Result ```

In [25]:
get_attribute_gini_index(pd_data, 'income', 'class_buys_computer')

{'high': 0.44285714285714295,
 'high_low': 0.4583333333333333,
 'high_medium': 0.45,
 'low': 0.45,
 'medium': 0.4583333333333333,
 'medium_low': 0.44285714285714295}

- 여기서 가장 작은 Gini index값을 가지는 class를 확인합니다.

In [26]:
min(get_attribute_gini_index(pd_data, 'income', 'class_buys_computer').items())

('high', 0.44285714285714295)

## 분류를 하는 데 가장 중요한 변수를 선정하고, 해당 변수의 Gini index를 제시해주세요.
- 모든 변수에 대한 Gini index(최소)를 출력해주세요.
- 해당 결과는 아래와 같이 나와야 합니다.

```Testing```

In [27]:
pd_data.columns[:-1]

Index(['age', 'income', 'student', 'credit_rating'], dtype='object')

In [28]:
min(get_attribute_gini_index(pd_data, 'age', 'class_buys_computer').items())

('middle_aged', 0.35714285714285715)

In [29]:
min(get_attribute_gini_index(pd_data, 'age', 'class_buys_computer').items())[1]

0.35714285714285715

```Result```

In [30]:
# 변수명 중 마지막에 위치한 label 컬럼 얻기
label = pd_data.columns[-1]
# label 변수를 제외한 변수명 얻기
features = list(pd_data.columns[:-1])

# 각 변수를 대상으로 반복문 수행(해당 변수 중 가장 낮은 gini 계수와 변수 출력)
for feature in features:
    
    min_gini = min(get_attribute_gini_index(pd_data, feature, label).items())[1]
    print(f"Minimum Gini index of {feature} : {min_gini :.4f}")

Minimum Gini index of age : 0.3571
Minimum Gini index of income : 0.4429
Minimum Gini index of student : 0.3673
Minimum Gini index of credit_rating : 0.4286


gini index가 가장 작게 나온 'age'를 가장 중요한 변수로 선정합니다.

이어서 해당 변수의 이진 분류된 각 class에 대해 Gini index도 계산합니다.

In [31]:
get_attribute_gini_index(pd_data, 'age', 'class_buys_computer')

{'middle_aged': 0.35714285714285715,
 'middle_aged_senior': 0.3936507936507937,
 'senior': 0.45714285714285713,
 'youth': 0.3936507936507937,
 'youth_middle_aged': 0.45714285714285713,
 'youth_senior': 0.35714285714285715}

'age' 변수에서 gini index가 가장 작게 나온 'middle_aged' class를 선정합니다.

## 위에서 선정한 feature로 DataFrame을 split한 후 나눠진 2개의 DataFrame에서 각각 다음으로 중요한 변수를 선정하고 해당 변수의 Gini index를 제시해주세요.

In [32]:
#선정한 feature로 데이터프레임 split을 해주세요.

df_1 = pd_data[pd_data['age'] == 'middle_aged']
df_2 = pd_data[pd_data['age'] != 'middle_aged']

In [33]:
df_1.head() #split 결과 확인 

,age,income,student,credit_rating,class_buys_computer
2,middle_aged,high,no,fair,yes
6,middle_aged,low,yes,excellent,yes
11,middle_aged,medium,no,excellent,yes
12,middle_aged,high,yes,fair,yes


In [34]:
df_2.head() #split 결과 확인

,age,income,student,credit_rating,class_buys_computer
0,youth,high,no,fair,no
1,youth,high,no,excellent,no
3,senior,medium,no,fair,yes
4,senior,low,yes,fair,yes
5,senior,low,yes,excellent,no


In [35]:
# 각 변수를 대상으로 반복문 수행
# 결과는 소수점 이하 4자리까지 출력

# 이미 나눠준 변수인 'age'를 제외한 나머지 변수들 할당
rest_features = ['income', 'student', 'credit_rating']

# Age 가 middle_aged인 데이터 셋에 적용 --> 변수들의 가장 낮은 지니계수
for feature in rest_features:
    print("Age : Middle_aged,", "Minimum Gini index of", feature, ":", round(min(get_attribute_gini_index(df_1, feature, label).values()), 4))

print("#############################################################")

# Age 가 middle_aged가 아닌 데이터 셋에 적용 --> 변수들의 가장 낮은 지니계수
for feature in rest_features:
    print("Age : NOT Middle_aged,", "Minimum Gini index of", feature, ":", round(min(get_attribute_gini_index(df_2, feature, label).values()), 4))

Age : Middle_aged, Minimum Gini index of income : 0.0
Age : Middle_aged, Minimum Gini index of student : 0.0
Age : Middle_aged, Minimum Gini index of credit_rating : 0.0
#############################################################
Age : NOT Middle_aged, Minimum Gini index of income : 0.375
Age : NOT Middle_aged, Minimum Gini index of student : 0.32
Age : NOT Middle_aged, Minimum Gini index of credit_rating : 0.4167


## Entropy 를 구하는 함수 만들기

<img src = https://miro.medium.com/max/1122/0*DkWdyGidNSfdT1Nu.png width = "350">

In [36]:
from math import log2

def getEntropy(df, feature) :
    
    # 엔트로피 값 초기 세팅
    entropy = 0
    # Feature의 unique한 value 추출한 뒤 리스트로 저장
    unique = list(df[feature].unique())     
    
    for i in unique:
        # p : 해당 Feature가 특정 값(i)을 가지는 데이터 개수 / Feature의 전체 데이터 개수
        p = sum(df[feature] == i) / len(df[feature])
        # - p * np.log2(p) 를 초기값 0에서 순차적으로 빼줌
        entropy -= p * np.log2(p)
        
    return(entropy)

In [37]:
getEntropy(pd_data, "class_buys_computer")

0.9402859586706309

In [38]:
# 가장 중요한 변수로 선정된 목표변수를 제외한 다른 변수들에 대해
# 각 칼럼별로 엔트로피를 구해주는 함수를 작성해주세요.

def getGainA(df, feature) :
        
    # 결과값을 담을 변수 초기 세팅
    result = {}

    # 타겟 변수 엔트로피 구하기
    info_D = getEntropy(df, feature)
    # 타겟 변수 제외한 나머지 변수들 리스트 형태로 저장
    columns = list(df.loc[:, df.columns != feature])

    for i in columns:
        # 가중치 초기 세팅
        w = 0 
        for j in list(df[i].unique()):
            S = len(df[feature])     # S : 전체 데이터 개수
            S_j = sum(df[i] == j)     # S_j : 전체 데이터 중 i 컬럼의 값이 j인 데이터의 개수 (= 가중치)
            w += (S_j / S) * getEntropy(df[df[i] == j], feature)   # 가중치 * Entropy 누적합 저장
        result[i] = info_D - w   # 최종 결과는 목표변수의 Entropy - w
        
    return(result)

In [39]:
getGainA(pd_data, "class_buys_computer")

{'age': 0.2467498197744391,
 'credit_rating': 0.04812703040826927,
 'income': 0.029222565658954647,
 'student': 0.15183550136234136}

Information Gain 값이 가장 큰 'age' 변수 채택